In [1]:
from __future__ import print_function

"""
A script to run cqed_rhf method on the formaldehyde molecule in a cc-pVDZ basis set.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from helper_cs_cqed_cis import *
from matplotlib import pyplot as plt

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2


First get the orbitals with full $C_{2v}$ symmetry using canonical RHF

In [3]:

# rhf/cc-pVDZ optimized geometry of formaldehyde
molstr = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c2v
"""

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-10,
               'd_convergence' : 1e-10}

psi4.set_options(options_dict)
mol = psi4.geometry(molstr)


energy, wfn = psi4.energy("scf/cc-pVDZ", molecule=mol, return_wfn=True)
print(energy)

# homo
HOMO = 8
# lumo+1
LUMOp1 = 10     # These are is the HOMO and LUMO+1 for formaldehyde




#psi4.set_options({
#    'CUBEPROP_TASKS': ['ORBITALS', 'DENSITY'], # calculate orbitals
#    'CUBEPROP_ORBITALS': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],    # calculate HOMO and LUMO. 
#})
psi4.set_options({
    'CUBEPROP_TASKS': ['ORBITALS'], # calculate orbitals
    'CUBEPROP_ORBITALS': [HOMO, LUMOp1],
})
cube = psi4.cubeprop(wfn)
import fortecubeview
fortecubeview.plot(colorscheme='emory', opacity=.95)

-113.87722217661772
CubeViewer: listing cube files from the directory .
Reading 2 cube files


/opt/anaconda3/lib//python3.8/site-packages/fortecubeview/py3js_renderer.py:1096: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  values = skimage.measure.marching_cubes_lewiner(data, level * 0.995)


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('MO    8a (2-B2)', 'MO   10a (6-A1)'), value=…

Now get the orbitals when there is a photon polarized along the $y-z$ axis with coupling
vector $\lambda_{yz} = \left(0, \frac{0.2}{\sqrt{2}}, \frac{0.2}{\sqrt{2}} \right) $.
Since our CQED-RHF code does not make use of symmetry, we will set the point group to be $C_1$, even
though in reality it is $C_s$ with this polarization.

In [4]:

# rhf/cc-pVDZ optimized geometry of formaldehyde
molstr = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1
"""

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-10,
               'd_convergence' : 1e-10}

psi4.set_options(options_dict)
mol = psi4.geometry(molstr)


energy, wfn = psi4.energy("scf/cc-pVDZ", molecule=mol, return_wfn=True)

# different magnitudes of the lambda vector
l = 0.2


lam_yz = np.array([0, l/np.sqrt(2), l/np.sqrt(2)])

yz_dict = cqed_rhf(lam_yz, molstr, options_dict)

C_yz = yz_dict['CQED-RHF C']

# store wfn as dictionary
rhf_wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# swap out orbitals
rhf_wfn_dict['matrix']['Ca'] = C_yz
rhf_wfn_dict['matrix']['Cb'] = C_yz

# Now create a new wavefunction object that has the CQED-RHF orbitals 'cqedrhf_wfn'
cqedrhf_wfn = psi4.core.Wavefunction.from_file(rhf_wfn_dict) 
# Confirm you have copied the CQED-RHF orbitals properly 
# by again getting a dic of the wavefunction and comparing
# the orbitals to the original numpy array 'C' that resulted
# from the CQED-RHF iterations
cqedrhf_wfn_dict = psi4.core.Wavefunction.to_file(cqedrhf_wfn)

assert np.allclose(cqedrhf_wfn_dict['matrix']['Ca'], C_yz, 1e-6)



# homo
HOMO = 8
# lumo+1
LUMOp1 = 10     # These are is the HOMO and LUMO+1 for formaldehyde


psi4.set_options({
    'CUBEPROP_TASKS': ['ORBITALS'], # calculate orbitals
    'CUBEPROP_ORBITALS': [HOMO, LUMOp1],
})
cube = psi4.cubeprop(cqedrhf_wfn)
import fortecubeview
fortecubeview.plot(colorscheme='emory', opacity=.95)


Start SCF iterations:

Canonical RHF One-electron energy = -218.2835959999194131
CQED-RHF One-electron energy      = -217.8061796768820386
Nuclear repulsion energy          = 31.7803657982661569
Dipole energy                     = 0.0142623507570482
SCF Iteration   1: Energy = -113.7228685077951269   dE = -1.13723E+02   dRMS = 2.16876E-03
SCF Iteration   2: Energy = -113.7282528209207300   dE = -5.38431E-03   dRMS = 8.46355E-04
SCF Iteration   3: Energy = -113.7286889666430909   dE = -4.36146E-04   dRMS = 4.27413E-04
SCF Iteration   4: Energy = -113.7287812472690831   dE = -9.22806E-05   dRMS = 3.23115E-04
SCF Iteration   5: Energy = -113.7288131295568689   dE = -3.18823E-05   dRMS = 2.18596E-04
SCF Iteration   6: Energy = -113.7288271434591280   dE = -1.40139E-05   dRMS = 1.67272E-04
SCF Iteration   7: Energy = -113.7288339957377872   dE = -6.85228E-06   dRMS = 1.18511E-04
SCF Iteration   8: Energy = -113.7288375348278322   dE = -3.53909E-06   dRMS = 8.95995E-05
SCF Iteration   9: En

/opt/anaconda3/lib//python3.8/site-packages/fortecubeview/py3js_renderer.py:1096: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  values = skimage.measure.marching_cubes_lewiner(data, level * 0.995)


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('MO    8a (2-B2)', 'MO    8a (8-A)', 'MO   10…